In [1]:
from pathlib import Path
import pandas as pd

In [2]:
folder_path = Path('data')
zip_files = list(folder_path.glob('*.zip'))
print(zip_files)
zip_file = zip_files[-1].absolute()
print(zip_file)

[WindowsPath('data/2023-04-26_00-54-lenta.zip'), WindowsPath('data/2023-04-27_14-27-lenta.zip'), WindowsPath('data/2023-04-30_13-43-lenta.zip'), WindowsPath('data/2023-05-07_21-14-lenta.zip'), WindowsPath('data/2023-05-13_20-33-lenta.zip'), WindowsPath('data/2023-05-19_23-28-lenta.zip'), WindowsPath('data/2023-06-02_12-55-lenta.zip'), WindowsPath('data/2023-06-09_13-46-lenta.zip'), WindowsPath('data/2023-06-24_16-23-lenta.zip')]
f:\works\work_002\data\2023-06-24_16-23-lenta.zip


In [3]:
df = pd.read_csv(zip_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28561 entries, 0 to 28560
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Категория      28561 non-null  object 
 1   Название       28561 non-null  object 
 2   Страна         28440 non-null  object 
 3   Скидка         15044 non-null  object 
 4   Цена           28561 non-null  float64
 5   Цена по карте  28544 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.3+ MB


In [4]:
mask = df['Название'].str.lower().str.contains(r'(?=молоко)(?=.* пастериз)(?=.*без змж)', regex=True)
find = df[mask].copy()
mask2 = find['Категория'] == 'Бакалея'
mask3 = find['Название'].str.lower().str.contains('лента')
# test = kilka[mask2].sort_values(by='Цена').copy()
test = find.copy()
test['volume'] = test['Страна'].str.extract('(\d+)').astype(int)
test['Пр. цена'] = round((test['Цена по карте'] / (test['volume'] / 1000)), 2)
mask4 = test['volume']>600
test_min_mean = test[mask4]['Пр. цена'].nsmallest(5).mean()
test_median = test[mask4]['Пр. цена'].median()
test_max_mean = test[mask4]['Пр. цена'].nlargest(5).mean()
print(f'минимум \t{test_min_mean:.2f}')
print(f'медиана \t{test_median:.2f}')
print(f'максимум \t{test_max_mean:.2f}')
print(f'количество позиций: {test.shape[0]}')
display(test.nsmallest(10, columns='Пр. цена'))

минимум 	64.97
медиана 	81.04
максимум 	128.88
количество позиций: 38


,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
1508,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 3,2%, бе...","Россия, 900 г",-49%,88.39,44.99,900,49.99
1923,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕНТА 2,5%, без змж, 900мл","Россия, 900 мл",-26%,81.09,59.99,900,66.66
1439,"Молоко, сыр, яйцо","Молоко пастеризованное СЧАСТЛИВОЕ ДЕТСТВО 1,5%...","Россия, 900 мл",NaN,64.79,61.49,900,68.32
2344,"Молоко, сыр, яйцо","Молоко пастеризованное ПРОСТОКВАШИНО 2,5%, без...","Россия, 930 мл",-28%,90.59,64.99,930,69.88
1488,"Молоко, сыр, яйцо","Молоко пастеризованное ЭКОНИВА 2,5%, без змж, ...","Россия, 1000 мл",-21%,89.49,69.99,1000,69.99
1629,"Молоко, сыр, яйцо","Молоко пастеризованное ПРАВИЛЬНОЕ МОЛОКО 2,5%,...","Россия, 2000 мл",-21%,178.99,139.99,2000,70.00
1490,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 2,5%, бе...","Россия, 900 г",NaN,67.19,63.79,900,70.88
1566,"Молоко, сыр, яйцо","Молоко пастеризованное BONVIDA 2,5%, без змж, ...","Россия, 1400 мл",-20%,126.39,99.99,1400,71.42
2467,"Молоко, сыр, яйцо","Молоко пастеризованное БЕЖИН ЛУГ 2,5%, без змж...","Россия, 1400 г",-26%,136.89,99.99,1400,71.42
1469,"Молоко, сыр, яйцо","Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО 3,2%, бе...","Россия, 900 г",NaN,69.49,65.99,900,73.32


In [5]:
display(test[test['volume']>600].nlargest(5, columns='Пр. цена'))

,Категория,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
2623,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,164.29,155.99,900,173.32
1759,"Молоко, сыр, яйцо",Молоко пастеризованное PARMALAT Comfort безлак...,"Россия, 900 мл",NaN,147.39,139.99,900,155.54
2477,"Молоко, сыр, яйцо","Молоко пастеризованное ВКУСНОТЕЕВО цельное 3,5...","Россия, 900 г",NaN,105.29,99.99,900,111.10
1649,"Молоко, сыр, яйцо",Молоко пастеризованное ЛЕБЕДЯНЬМОЛОКО отборное...,"Россия, 900 г",NaN,102.19,96.99,900,107.77
1575,"Молоко, сыр, яйцо",Молоко пастеризованное ВКУСНОТЕЕВО ультрачисто...,"Россия, 900 г",NaN,91.59,86.99,900,96.66


In [6]:
choc_mask_01 = df['Название'].str.lower().str.contains(r'(?=.*шоколад)(?=.*белый)', regex=True)
choc_mask_02 = df['Категория'] == 'Кондитерские изделия'
chocolate = df[choc_mask_01 & choc_mask_02].copy().reset_index(drop='index')
chocolate['volume'] = chocolate['Страна'].str.extract('(\d+)').astype(int)
chocolate['Пр. цена'] = round((chocolate['Цена по карте'] / (chocolate['volume'] / 100)), 2)
chocolate[chocolate.columns[1:]].sort_values('Пр. цена')

,Название,Страна,Скидка,Цена,Цена по карте,volume,Пр. цена
0,"Шоколад белый ALPEN GOLD с миндалем и кокосом,...","Россия, 85 г",-52%,105.29,49.99,85,58.81
5,Шоколад молочный и белый РОССИЯ ЩЕДРАЯ ДУША с ...,"Россия, 82 г",-59%,123.56,49.99,82,60.96
3,Шоколад молочный и белый РОССИЯ ЩЕДРАЯ ДУША по...,"Россия, 75 г",-60%,126.96,49.99,75,66.65
8,Плитка шоколадная DR. CHOQ шоколад белый с мер...,"Бельгия, 150 г",-27%,178.99,129.99,150,86.66
6,"Шоколад белый ОСОБЫЙ Карамельный с солью, порц...","Россия, 88 г",-54%,175.16,79.99,88,90.90
2,"Шоколад белый пористый ВОЗДУШНЫЙ, 85г","Россия, 85 г",-42%,139.34,79.99,85,94.11
4,"Шоколад KDV BabyFox с малиной, молочный и белы...","Россия, 90 г",NaN,105.29,99.99,90,111.10
1,Шоколад белый RITTER SPORT Цельный лесной орех...,"Германия, 100 г",-38%,194.79,119.99,100,119.99
9,Шоколад белый А&А ДЕМИДОВ с добавлением кусочк...,"Россия, 100 г",NaN,152.69,144.99,100,144.99
7,Шоколад белый РОССИЯ ЩЕДРАЯ ДУША Gold Selectio...,"Россия, 85 г",NaN,129.99,123.49,85,145.28


In [7]:
import requests
from bs4 import BeautifulSoup as BS
import lxml
from pprint import pprint

url = 'https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D1%82%D1%80%D0%B5%D0%B1%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F_%D0%BA%D0%BE%D1%80%D0%B7%D0%B8%D0%BD%D0%B0'

response = requests.get(url)
soup = BS(response.content, 'lxml')

table = soup.find('table', class_='wikitable')
rows = list(table.find_all('tr'))[2:]

sum_list = []
for row in rows[:]:
    out_list = []
    for index, element in enumerate(row.find_all('td')):
        if index > 1:
            per_year = float(element.get_text(strip=True).replace(',', '.'))
            per_month = per_year / 12
            out_list.append(round(per_month, 2))
        else:
            out_list.append(element.get_text(strip=True))
    sum_list.append(out_list)

table_df = pd.DataFrame(sum_list, columns=['name', 'unit', 'works', 'pensioners', 'children'])
table_df

,name,unit,works,pensioners,children
0,Хлебные продукты (хлебимакаронные изделияв пер...,кг,10.54,8.18,6.47
1,Картофель,кг,8.37,6.67,7.34
2,Овощиибахчевые,кг,9.55,8.17,9.38
3,Фруктысвежие,кг,5.00,3.75,9.84
4,Сахарикондитерские изделияв пересчете на сахар,кг,1.98,1.77,1.82
5,Мясопродукты,кг,4.88,4.50,3.67
6,Рыбопродукты,кг,1.54,1.33,1.55
7,Молокои молокопродукты в пересчете на молоко,кг,24.17,21.48,30.06
8,Яйца,штука,17.50,16.67,16.75
9,"Масло растительное,маргарини другиежиры",кг,0.92,0.83,0.42


In [8]:
mask = df['Название'].str.lower().str.contains(r'(?=.*стиральный.*)(?=.*порошок.*)', regex=True)
df_test = pd.DataFrame(df[mask].reset_index(drop='index'))
# mask2 = df_test['Название'].str.lower().str.contains(r'(?=.*)', regex=True)
df_test['Вес'] = df_test['Страна'].apply(lambda x: ''.join(x.split(',', maxsplit=1)[1:]))
df_test['Вес'] = df_test['Вес'].str.extract('(\d+\,?\d*)')
df_test['Вес'] = df_test['Вес'].str.replace(',', '.')
df_test['Вес'] = df_test['Вес'].astype(float).apply(lambda x: x if x > 100 else x * 1000).astype(int)

df_test['Страна'] = df_test['Страна'].apply(lambda x: x.split(',')[0])
# print(df_test['Вес'].unique())
df_test['За кг'] = df_test['Цена по карте'] / df_test['Вес'] * 1000
df_test['За кг'] = df_test['За кг'].round().astype(int)
# df_test[df_test['Страна'] == 'Дания']
df_test[~(df_test['Страна'] == 'Россия')].nsmallest(n=20, columns='За кг')

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer